### Load the model and make prediction
##### Input Args:

doc_dir = directory where the file with comments to be predicted

doc2vec_model_name = doc2vec model already trained

trained_clf = Trained classifier


In [1]:
from gensim.models import Doc2Vec
import re
import os
import random
from sklearn import metrics
import numpy as np
import pandas as pd
from sklearn.externals import joblib
from sklearn.metrics.pairwise import cosine_similarity
import pickle

In [2]:
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

In [3]:
def extract_words(sent):
    sent = sent.lower()
    sent = re.sub(r'<[^>]+>', ' ', sent) #strip html tags
    sent = re.sub(r'(\w)\'(\w)', '\1\2', sent) # remoce apostrophes
    sent = re.sub(r'\W',' ', sent) # remove punctuation
    sent = re.sub(r'\s+', ' ', sent) # remove repeated space
    sent = sent.strip()
    return sent.split()

In [4]:
#Load doc to vec model
doc2vec_model_name = 'reviews.d2v'
doc2vec_model = Doc2Vec.load(doc2vec_model_name)

In [5]:
#Test the trained doc2vec model
cosine_similarity(
[doc2vec_model.infer_vector(extract_words("This is very bad video. I don't like it"))],
[doc2vec_model.infer_vector(extract_words("video sucks."))])

array([[0.7344312]], dtype=float32)

In [6]:
#Converts the comments to vector using doc2vec model trianed earlier
def get_doc2vec(model, doc_dir,comment_col_name='Comment'):
    comments_dfs = []
    
    for fname in sorted(os.listdir(doc_dir)):
        fname_full = doc_dir + "/" + fname
        print("files being read : {0}".format(fname_full))
        data = pd.read_csv(fname_full, sep='\t', header='infer')
        data['FNAME'] = fname
        data['WORDS'] = data[comment_col_name].map(lambda x: extract_words(x))
        data['SENTVECS'] = data['WORDS'].map(lambda x: model.infer_vector(x, steps=10))
        comments_dfs.append(data)

    comments = pd.concat(comments_dfs)
    return comments

In [7]:
doc_dir = "../indata"
comments = get_doc2vec(doc2vec_model, doc_dir)

files being read : ../indata/eminem-not-affraid.csv
files being read : ../indata/got8.csv
files being read : ../indata/house-of-cards.csv
files being read : ../indata/samsung-vr.csv
files being read : ../indata/tonight-show.csv
files being read : ../indata/tswift.csv
files being read : ../indata/warriors.csv


In [8]:
comments.head(5)

,Comment,CreateTimeStamp,Type,videoID,FNAME,WORDS,SENTVECS
0,Quien escucha en,2018-12-02T02:52:14.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[quien, escucha, en]","[-0.01628579, 0.11731559, -0.09368828, 0.05932..."
1,I m not afraid ok Oscar the grouch go chill on...,2018-12-02T02:32:31.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[i, m, not, afraid, ok, oscar, the, grouch, go...","[0.4212175, -0.038643703, 0.49855557, -0.21210..."
2,Eminemcia,2018-12-02T02:03:50.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,[eminemcia],"[0.00032911325, 0.0073782154, -0.005224751, 0...."
3,My favorite rapper of all time,2018-12-02T02:02:17.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[my, favorite, rapper, of, all, time]","[0.032914963, 0.26085487, 0.08095396, -0.27867..."
4,He was probably shitting his pants on top of t...,2018-12-02T01:59:49.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[he, was, probably, shitting, his, pants, on, ...","[-0.18870537, 0.31686017, 0.07021599, -0.14053..."


In [9]:
print("No of files : {0}".format(comments.FNAME.unique()))
print("No of Type : {}".format(comments.Type.unique()))

No of files : ['eminem-not-affraid.csv' 'got8.csv' 'house-of-cards.csv' 'samsung-vr.csv'
 'tonight-show.csv' 'tswift.csv' 'warriors.csv']
No of Type : ['Comment' 'Reply']


In [10]:
trained_clf = 'doc2vec_spam_rf.model'
loaded_clf = joblib.load(trained_clf)

/anaconda/lib/python3.6/site-packages/sklearn/ensemble/weight_boosting.py:29: DeprecationWarning: numpy.core.umath_tests is an internal NumPy module and should not be imported. It will be removed in a future NumPy release.
  from numpy.core.umath_tests import inner1d


In [11]:
comments['SPAM_IND_RF'] = loaded_clf.predict(comments['SENTVECS'].tolist())

In [12]:
comments.head(10)

,Comment,CreateTimeStamp,Type,videoID,FNAME,WORDS,SENTVECS,SPAM_IND_RF
0,Quien escucha en,2018-12-02T02:52:14.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[quien, escucha, en]","[-0.01628579, 0.11731559, -0.09368828, 0.05932...",0
1,I m not afraid ok Oscar the grouch go chill on...,2018-12-02T02:32:31.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[i, m, not, afraid, ok, oscar, the, grouch, go...","[0.4212175, -0.038643703, 0.49855557, -0.21210...",1
2,Eminemcia,2018-12-02T02:03:50.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,[eminemcia],"[0.00032911325, 0.0073782154, -0.005224751, 0....",0
3,My favorite rapper of all time,2018-12-02T02:02:17.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[my, favorite, rapper, of, all, time]","[0.032914963, 0.26085487, 0.08095396, -0.27867...",0
4,He was probably shitting his pants on top of t...,2018-12-02T01:59:49.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[he, was, probably, shitting, his, pants, on, ...","[-0.18870537, 0.31686017, 0.07021599, -0.14053...",0
5,December Anyone,2018-12-02T01:55:21.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[december, anyone]","[-0.031744253, 0.08422521, 0.031333838, -0.108...",0
6,Best of Best,2018-12-02T01:47:07.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[best, of, best]","[-0.15499787, 0.08648931, 0.18655306, -0.10343...",0
7,Who watch emiway bantai tribute to eminem anyo...,2018-12-02T01:46:34.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[who, watch, emiway, bantai, tribute, to, emin...","[0.40776566, 0.3263037, 0.24301608, -0.1800967...",0
8,Wow is Rap,2018-12-02T01:10:13.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[wow, is, rap]","[0.053112626, 0.23462449, 0.08876002, -0.16443...",0
9,Poland k,2018-12-02T01:09:36.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[poland, k]","[0.028835002, -0.097403355, -0.2357736, -0.017...",0


In [13]:
trained_clf = 'doc2vec_spam_kn.model'
loaded_clf = joblib.load(trained_clf)

In [14]:
comments['SPAM_IND_KN'] = loaded_clf.predict(comments['SENTVECS'].tolist())

In [15]:
comments.head(10)

,Comment,CreateTimeStamp,Type,videoID,FNAME,WORDS,SENTVECS,SPAM_IND_RF,SPAM_IND_KN
0,Quien escucha en,2018-12-02T02:52:14.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[quien, escucha, en]","[-0.01628579, 0.11731559, -0.09368828, 0.05932...",0,0
1,I m not afraid ok Oscar the grouch go chill on...,2018-12-02T02:32:31.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[i, m, not, afraid, ok, oscar, the, grouch, go...","[0.4212175, -0.038643703, 0.49855557, -0.21210...",1,1
2,Eminemcia,2018-12-02T02:03:50.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,[eminemcia],"[0.00032911325, 0.0073782154, -0.005224751, 0....",0,0
3,My favorite rapper of all time,2018-12-02T02:02:17.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[my, favorite, rapper, of, all, time]","[0.032914963, 0.26085487, 0.08095396, -0.27867...",0,1
4,He was probably shitting his pants on top of t...,2018-12-02T01:59:49.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[he, was, probably, shitting, his, pants, on, ...","[-0.18870537, 0.31686017, 0.07021599, -0.14053...",0,0
5,December Anyone,2018-12-02T01:55:21.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[december, anyone]","[-0.031744253, 0.08422521, 0.031333838, -0.108...",0,0
6,Best of Best,2018-12-02T01:47:07.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[best, of, best]","[-0.15499787, 0.08648931, 0.18655306, -0.10343...",0,0
7,Who watch emiway bantai tribute to eminem anyo...,2018-12-02T01:46:34.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[who, watch, emiway, bantai, tribute, to, emin...","[0.40776566, 0.3263037, 0.24301608, -0.1800967...",0,1
8,Wow is Rap,2018-12-02T01:10:13.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[wow, is, rap]","[0.053112626, 0.23462449, 0.08876002, -0.16443...",0,0
9,Poland k,2018-12-02T01:09:36.000Z,Comment,j5-yKhDd64s,eminem-not-affraid.csv,"[poland, k]","[0.028835002, -0.097403355, -0.2357736, -0.017...",0,0


In [16]:
out_df = comments[['Comment','CreateTimeStamp','Type', 'videoID','FNAME', 'SPAM_IND_RF','SPAM_IND_KN']]

In [17]:
out_df.head(5)

,Comment,CreateTimeStamp,Type,videoID,SPAM_IND_RF,SPAM_IND_KN
0,Quien escucha en,2018-12-02T02:52:14.000Z,Comment,j5-yKhDd64s,0,0
1,I m not afraid ok Oscar the grouch go chill on...,2018-12-02T02:32:31.000Z,Comment,j5-yKhDd64s,1,1
2,Eminemcia,2018-12-02T02:03:50.000Z,Comment,j5-yKhDd64s,0,0
3,My favorite rapper of all time,2018-12-02T02:02:17.000Z,Comment,j5-yKhDd64s,0,1
4,He was probably shitting his pants on top of t...,2018-12-02T01:59:49.000Z,Comment,j5-yKhDd64s,0,0


In [18]:
out_file_name = '../outdata/video_comments_analysis_spam.csv'
out_df.to_csv(out_file_name, sep='\t', index=False)

In [19]:
print('****End of script****')

****End of script****
